# Natural Language Processing Code Along Project.

### (Henri's Version)

Let's work through building a spam detection filter using Python and Spark.

Our data set consists of volunteered text messages from a study in Singapore and some spam text from a UK reporting site.

Let's get started.

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("spam_vs_ham").getOrCreate()

In [3]:
data = spark.read.csv("SMSSpamCollection", inferSchema=True, sep="\t")
# there is no header, and the fields are separated by tabs.

In [4]:
data.show()
# Notice how our header column names default to _c0 and _c1

+----+--------------------+
| _c0|                 _c1|
+----+--------------------+
| ham|Go until jurong p...|
| ham|Ok lar... Joking ...|
|spam|Free entry in 2 a...|
| ham|U dun say so earl...|
| ham|Nah I don't think...|
|spam|FreeMsg Hey there...|
| ham|Even my brother i...|
| ham|As per your reque...|
|spam|WINNER!! As a val...|
|spam|Had your mobile 1...|
| ham|I'm gonna be home...|
|spam|SIX chances to wi...|
|spam|URGENT! You have ...|
| ham|I've been searchi...|
| ham|I HAVE A DATE ON ...|
|spam|XXXMobileMovieClu...|
| ham|Oh k...i'm watchi...|
| ham|Eh u remember how...|
| ham|Fine if thats th...|
|spam|England v Macedon...|
+----+--------------------+
only showing top 20 rows



In [5]:
data = data.withColumnRenamed("_c0", "class").withColumnRenamed("_c1", "text")

In [6]:
data.show()

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
|  ham|I'm gonna be home...|
| spam|SIX chances to wi...|
| spam|URGENT! You have ...|
|  ham|I've been searchi...|
|  ham|I HAVE A DATE ON ...|
| spam|XXXMobileMovieClu...|
|  ham|Oh k...i'm watchi...|
|  ham|Eh u remember how...|
|  ham|Fine if thats th...|
| spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows



### Clean and prepare our data using all of the tools that we learned in the previous lectures.

In [7]:
from pyspark.sql.functions import length

In [8]:
# Create a new column called "length".  Use the length function of pyspark.sql.functions
data = data.withColumn("length", length(data["text"]))
# It counts the number of characters, not the number of words.

In [9]:
data.show()

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



In [10]:
data.groupBy("class").mean().show()
# Average the numerical columns, and separate them by the class labels: ["ham", "spam"].

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



In [11]:
data.groupby("class").count().show()

+-----+-----+
|class|count|
+-----+-----+
|  ham| 4827|
| spam|  747|
+-----+-----+



In [12]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer, IDF, StringIndexer

In [13]:
tokenizer = Tokenizer(inputCol="text", outputCol="token_text")  # split into a list of words.
stop_remove = StopWordsRemover(inputCol="token_text", outputCol="stop_token")  # remove insignificant words.
count_vect = CountVectorizer(inputCol="stop_token", outputCol="c_vec")
idf = IDF(inputCol="c_vec", outputCol="tf_idf")
ham_spam_to_numeric = StringIndexer(inputCol="class", outputCol="label")

In [14]:
from pyspark.ml.feature import VectorAssembler

In [15]:
clean_up = VectorAssembler(inputCols=["tf_idf", "length"], outputCol="features")

In [16]:
from pyspark.ml.classification import NaiveBayes
# pyspark.ml.classification.NaiveBayes(featuresCol='features', labelCol='label', predictionCol='prediction', 
#                                      probabilityCol='probability', rawPredictionCol='rawPrediction', 
#                                      smoothing=1.0, modelType='multinomial', thresholds=None, weightCol=None)
# --> Naive Bayes Classifiers.
# It supports both Multinomial and Bernoulli NB.
# Multinomial NB can handle finitely supported discrete data. 
# For example, by converting documents into TF-IDF vectors, it can be used for document classification.
# By making every vector a binary (0/1) data, it can also be used as Bernoulli NB.
# The input feature values must be nonnegative.


# Naive Bayes is a natural choice when it comes to Natural Language Processing.

In [17]:
nb = NaiveBayes()  # use all the defaults.

In [18]:
from pyspark.ml import Pipeline

In [19]:
data_prep_pipe = Pipeline(stages=[ham_spam_to_numeric, 
                                  tokenizer,
                                  stop_remove,
                                  count_vect,
                                  idf,
                                  clean_up])

In [20]:
cleaner = data_prep_pipe.fit(data)

In [21]:
clean_data = cleaner.transform(data)

In [22]:
clean_data.columns

['class',
 'text',
 'length',
 'label',
 'token_text',
 'stop_token',
 'c_vec',
 'tf_idf',
 'features']

In [23]:
organised_data = clean_data.select(["features", "label"])

In [24]:
organised_data.show()

+--------------------+-----+
|            features|label|
+--------------------+-----+
|(13424,[7,11,31,6...|  0.0|
|(13424,[0,24,297,...|  0.0|
|(13424,[2,13,19,3...|  1.0|
|(13424,[0,70,80,1...|  0.0|
|(13424,[36,134,31...|  0.0|
|(13424,[10,60,139...|  1.0|
|(13424,[10,53,103...|  0.0|
|(13424,[125,184,4...|  0.0|
|(13424,[1,47,118,...|  1.0|
|(13424,[0,1,13,27...|  1.0|
|(13424,[18,43,120...|  0.0|
|(13424,[8,17,37,8...|  1.0|
|(13424,[13,30,47,...|  1.0|
|(13424,[39,96,217...|  0.0|
|(13424,[552,1697,...|  0.0|
|(13424,[30,109,11...|  1.0|
|(13424,[82,214,47...|  0.0|
|(13424,[0,2,49,13...|  0.0|
|(13424,[0,74,105,...|  0.0|
|(13424,[4,30,33,5...|  1.0|
+--------------------+-----+
only showing top 20 rows



In [25]:
train_data, test_data = organised_data.randomSplit([0.7, 0.3], seed=123)

In [26]:
spam_detector = nb.fit(train_data)

In [27]:
test_results = spam_detector.transform(dataset=test_data)

In [28]:
test_results.select(["label", "prediction"]).show()

+-----+----------+
|label|prediction|
+-----+----------+
|  1.0|       1.0|
|  1.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  1.0|       1.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
+-----+----------+
only showing top 20 rows



### Use some evaluation metrics to compare ```label``` with ```prediction```.

In [29]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [30]:
# Contrary to the instructor's method of using the accuracy metric, I am going to use the f1 and precision metrics
# I think the accuracy metric is a poor choice since there are 4827 ham instances, and 747 spam instances, 
# which is rather unbalanced.
f1_eval = MulticlassClassificationEvaluator(metricName="f1")
prec_eval = MulticlassClassificationEvaluator(metricName="weightedPrecision")

In [31]:
f1_score = f1_eval.evaluate(test_results)
prec_score = prec_eval.evaluate(test_results)

In [32]:
print("F1 score: {:.3f}.".format(f1_score))
print("Precision: {:.3f}.".format(prec_score))

F1 score: 0.925.
Precision: 0.947.
